In [1]:
import pandas as pd
import numpy as np
from bs4 import BeautifulSoup
import requests

## Part 1

#### Download neigbourhood table from Wiki and store to a pandas dataframe

In [2]:
# Download the neigbourhood table from Wiki
source = requests.get('https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M').text

# Parse the html table
soup = BeautifulSoup (source, 'html.parser')
code_table = soup.find('table', class_='wikitable sortable')

# Convert html table to dataframe
df = pd.read_html(str(code_table))[0]

#### Drop rows with Borough == 'Not assigned'

In [3]:
df.drop(df[df.Borough == 'Not assigned'].index, inplace=True)

#### Concatenate neigbourhoods grouped by borough

In [4]:
df = df.groupby(['Postcode','Borough'])['Neighbourhood'].apply(', '.join).reset_index()

#### If a cell has a borough but a Not assigned neighborhood, then the neighborhood will be the same as the borough

In [5]:
df.loc[df.Neighbourhood == 'Not assigned', 'Neighbourhood'] = df.Borough
# Test if any Neighbourhood still have value 'Not assigned' - should return empty table
df[df.Neighbourhood == 'Not assigned']

,Postcode,Borough,Neighbourhood


#### Print shape of the resulting dataframe

In [6]:
df.shape

(103, 3)

In [7]:
# Rename column Postcode to PostalCode
df.rename(index=str, columns={"Postcode": "PostalCode"}, inplace=True)

In [8]:
df.head()

,PostalCode,Borough,Neighbourhood
0,M1B,Scarborough,"Rouge, Malvern"
1,M1C,Scarborough,"Highland Creek, Rouge Hill, Port Union"
2,M1E,Scarborough,"Guildwood, Morningside, West Hill"
3,M1G,Scarborough,Woburn
4,M1H,Scarborough,Cedarbrae


## Part 2

#### Geocoder Python package is unreliable, load geographical coordinates from the provided csv instead 

In [9]:
csv_url="https://cocl.us/Geospatial_data"
coordinates_df=pd.read_csv(csv_url)

In [10]:
# Rename column "Postal Code" to "PostalCode"
coordinates_df.rename(index=str, columns={"Postal Code": "PostalCode"}, inplace=True)
coordinates_df.head()

,PostalCode,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


#### Join the neigbourhood table with coordinates

In [13]:
df = pd.merge(df, coordinates_df, on='PostalCode', how='left')

In [14]:
df.head()

,PostalCode,Borough,Neighbourhood,Latitude,Longitude
0,M1B,Scarborough,"Rouge, Malvern",43.806686,-79.194353
1,M1C,Scarborough,"Highland Creek, Rouge Hill, Port Union",43.784535,-79.160497
2,M1E,Scarborough,"Guildwood, Morningside, West Hill",43.763573,-79.188711
3,M1G,Scarborough,Woburn,43.770992,-79.216917
4,M1H,Scarborough,Cedarbrae,43.773136,-79.239476


## Part 3

In [39]:
# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans

import folium # map rendering library

#### Plot all neighbourhood on a map

In [59]:
latitude_Toronto = 43.72
longitude_Toronto = -79.3849

In [61]:
# create map of New York using latitude and longitude values
map_canada = folium.Map(location=[latitude_Toronto, longitude_Toronto], zoom_start=11)

# add markers to map
for lat, lng, borough, neighborhood in zip(df['Latitude'], df['Longitude'], df['Borough'], df['Neighbourhood']):
    label = '{}, {}'.format(neighborhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_canada)  
    
map_canada

In [32]:
df.Borough.unique()

array(['Scarborough', 'North York', 'East York', 'East Toronto',
       'Central Toronto', 'Downtown Toronto', 'York', 'West Toronto',
       "Queen's Park", 'Mississauga', 'Etobicoke'], dtype=object)

#### Create clustering features: using coordinates as features

In [69]:
# Create
df_clustering = df.drop(['Neighbourhood', 'PostalCode', 'Borough'], 1)

In [70]:
df_clustering.head()

,Latitude,Longitude
0,43.806686,-79.194353
1,43.784535,-79.160497
2,43.763573,-79.188711
3,43.770992,-79.216917
4,43.773136,-79.239476


#### Cluster neighbourhoods based on coordinates data

In [85]:
# set number of clusters
kclusters = 5

# Run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(df_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_

array([0, 0, 0, 0, 0, 0, 0, 2, 0, 2, 0, 0, 0, 0, 0, 0, 0, 4, 4, 4, 4, 4,
       4, 4, 4, 2, 2, 2, 4, 4, 4, 1, 1, 1, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2,
       4, 4, 4, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 4, 4, 3, 3,
       3, 3, 3, 3, 3, 4, 4, 3, 3, 3, 3, 3, 3, 1, 1, 1, 3, 3, 1, 3, 1, 2,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1])

In [ ]:
# Add clustering lables to neighbourhood table
df.insert(0, 'Cluster Labels', kmeans.labels_)

In [84]:
df.head()

,Cluster Labels,PostalCode,Borough,Neighbourhood,Latitude,Longitude
0,0,M1B,Scarborough,"Rouge, Malvern",43.806686,-79.194353
1,0,M1C,Scarborough,"Highland Creek, Rouge Hill, Port Union",43.784535,-79.160497
2,0,M1E,Scarborough,"Guildwood, Morningside, West Hill",43.763573,-79.188711
3,0,M1G,Scarborough,Woburn,43.770992,-79.216917
4,0,M1H,Scarborough,Cedarbrae,43.773136,-79.239476


#### Visualize clustered neighbourhoods on a map

In [92]:
# create map
map_clusters = folium.Map(location=[latitude_Toronto, longitude_Toronto], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(df['Latitude'], df['Longitude'], df['Neighbourhood'], df['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters